<a href="https://colab.research.google.com/github/qandeelfatima11/Job-Recommendation-System/blob/main/job_recommendation_system.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### This is Recommendation model which recommends jobs based upon your location

# Data Aquisition

In [ ]:
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
from selenium import webdriver
from time import sleep
from selenium.webdriver.chrome.options import Options
from selenium.webdriver import Chrome
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.keys import Keys


In [ ]:
from selenium import webdriver
import time
from selenium.webdriver.common.keys import Keys

s = Service('/usr/bin/chromedriver')
myoptions = Options()
myoptions.headless = True

driver = Chrome(service=s, options=myoptions)

driver.get('https://www.glassdoor.com/profile/login_input.htm')


# username = driver.find_element_by_name('username')
username = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.NAME, 'username')))
username.send_keys('msdsf22m006@pucit.edu.pk')
button_element = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.CSS_SELECTOR, ".Button.Button"))
    )
button_element.click()

# password = driver.find_element_by_name('password')
password = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.NAME, 'password')))
password.send_keys('12345678!')
driver.find_element(By.CLASS_NAME, 'gd-ui-button').click()

time.sleep(3)

# Navigate to the job search page

driver.get('https://www.glassdoor.com/Job/jobs.htm')


WebDriverWait(driver, 20).until(lambda driver: driver.execute_script("return document.readyState") == "complete")
search = WebDriverWait(driver, 20).until(
    EC.element_to_be_clickable((By.ID, 'searchBar-jobTitle'))
)

search.send_keys('data scientist')
location = driver.find_element(By.ID, 'searchBar-location')
location.clear()
location.send_keys('Pakistan')
location.send_keys(Keys.ENTER)
time.sleep(3)


jobs = driver.find_elements(By.CLASS_NAME, 'JobCard_jobCardContainer__l0svv')
data_list = []

time.sleep(3)
for job in jobs:
    title = (job.find_element(By.CLASS_NAME, 'JobCard_seoLink__WdqHZ').text)
    location = (job.find_element(By.CLASS_NAME,'JobCard_location__N_iYE').text)
    company = (job.find_element(By.CLASS_NAME,'EmployerProfile_employerInfo__GaPbq').text)

    data_dict = {
            'Title': title,
            'CompanyName': company,
            'Location': location
        }
    data_list.append(data_dict)

driver.quit()
df = pd.DataFrame(data_list)
df.head()

,Title,CompanyName,Location
0,Data Scientist / Strategist,"Lahore, Punjab",Pakistan
1,Data Scientist,Leverify,Pakistan
2,Data Scientist,Xcentric Services4 ★,Lahore
3,Data Scientist/ML Engineer,Zepto Systems Limited3.7 ★,Lahore
4,Data Engineer,AirCod Technologies,Lahore


In [ ]:
df.head(20)

,Title,CompanyName,Location
0,Data Scientist / Strategist,"Lahore, Punjab",Pakistan
1,Data Scientist,Leverify,Pakistan
2,Data Scientist,Xcentric Services4 ★,Lahore
3,Data Scientist/ML Engineer,Zepto Systems Limited3.7 ★,Lahore
4,Data Engineer,AirCod Technologies,Lahore
5,Senior Data Scientist,HR Force International,Lahore
6,AI/ML Expert,Punch Digital Agency3.7 ★,Lahore
7,Senior AI Engineer,Tekhqs,Lahore
8,Senior Data Scientist I,DevGate Consultancy,Islamabad
9,ML ops Intern,AxcelerateAI,Lahore


# PreProcessig

In [ ]:
df.describe()

,Title,CompanyName,Location
count,30,30,30
unique,25,29,5
top,Senior Data Scientist,Leverify,Lahore
freq,4,2,15


In [ ]:
df.isna().sum()

Title          0
CompanyName    0
Location       0
dtype: int64

Converting CompanyName into two columns

In [ ]:
import re

def extract_rating(text):
    rating = re.search(r'(\d+\.*\d*)\s*★', text)
    if rating:
        return float(rating.group(1))
    else:
        return None

#create a new 'rating' column
df['Rating'] = df['CompanyName'].apply(lambda x: extract_rating(x))

#extract company name
def extract_company(text):
    return re.sub(r'\d+\.*\d*\s*★', '', text).strip()

#create a new 'company_name' column
df['Company_name'] = df['CompanyName'].apply(lambda x: extract_company(x))

# Drop original column
df.drop(columns=['CompanyName'], inplace=True)

df.head()


,Title,Location,Rating,Company_name
0,Data Scientist / Strategist,Pakistan,NaN,"Lahore, Punjab"
1,Data Scientist,Pakistan,NaN,Leverify
2,Data Scientist,Lahore,4.0,Xcentric Services
3,Data Scientist/ML Engineer,Lahore,3.7,Zepto Systems Limited
4,Data Engineer,Lahore,NaN,AirCod Technologies


In [ ]:
df.isna().sum()

Title            0
Location         0
Rating          20
Company_name     0
dtype: int64

In [ ]:
most_common_location = df['Location'].mode()[0]
most_common_location

'Lahore'

Replacing location with 'Pakistan'

In [ ]:
df['Location'] = df['Location'].replace('', 'Pakistan')
df.head(10)

,Title,Location,Rating,Company_name
0,Data Scientist / Strategist,Pakistan,NaN,"Lahore, Punjab"
1,Data Scientist,Pakistan,NaN,Leverify
2,Data Scientist,Lahore,4.0,Xcentric Services
3,Data Scientist/ML Engineer,Lahore,3.7,Zepto Systems Limited
4,Data Engineer,Lahore,NaN,AirCod Technologies
5,Senior Data Scientist,Lahore,NaN,HR Force International
6,AI/ML Expert,Lahore,3.7,Punch Digital Agency
7,Senior AI Engineer,Lahore,NaN,Tekhqs
8,Senior Data Scientist I,Islamabad,NaN,DevGate Consultancy
9,ML ops Intern,Lahore,NaN,AxcelerateAI


replacing no rating with 0

In [ ]:
df['Rating'].fillna(0.0, inplace=True)
df.head()

,Title,Location,Rating,Company_name
0,Data Scientist / Strategist,Pakistan,0.0,"Lahore, Punjab"
1,Data Scientist,Pakistan,0.0,Leverify
2,Data Scientist,Lahore,4.0,Xcentric Services
3,Data Scientist/ML Engineer,Lahore,3.7,Zepto Systems Limited
4,Data Engineer,Lahore,0.0,AirCod Technologies


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel

# Combine location, company name, and title into a single feature for vectorization
df['Features'] = df['Location'] + ' ' + df['Company_name'] + ' ' + df['Title']

# Use TF-IDF vectorizer to convert text features into numerical vectors
tfidf_vectorizer = TfidfVectorizer()
tfidf_matrix = tfidf_vectorizer.fit_transform(df['Features'])

# Calculate cosine similarity between items (jobs) based on their features
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)

def get_top_n_recommendations(title, cosine_sim, df, n=5):
    # Convert input title to lowercase
    title = title.lower()

    # Convert all job titles in the DataFrame to lowercase for comparison
    df['Title'] = df['Title'].apply(lambda x: x.lower())

    # Check if the input title exists in the DataFrame
    if title not in df['Title'].values:
        print(f"Job title '{title}' not found.")
        return None

    # Get the index of the input title in the DataFrame
    idx = df[df['Title'] == title].index[0]
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:n+1]
    job_titles = [df['Title'].iloc[i[0]] for i in sim_scores]
    return job_titles

job_title = 'data engineer'
top_recommendations = get_top_n_recommendations(job_title, cosine_sim, df, n=10)

if top_recommendations is not None:
    print(f"Top recommendations for '{job_title}':")
    print(top_recommendations)
else:
    print("No recommendations available.")


Top recommendations for 'data scientist':
['machine learning engineer (cloud practitioner)', 'data scientist / strategist', 'senior data scientist', 'sr.data scientist', 'senior data scientist', 'senior data scientist', 'data scientist', 'senior data scientist i', 'data scientist', 'senior data scientist']


In [ ]:
import pickle

# Pickle the variable and save it to a file
with open('top_recommendations.pkl', 'wb') as file:
    pickle.dump(top_recommendations, file)
